# Exercise Session 3: Canny Edge Detection

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import cv2

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.cmap'] = 'gray'

## Q2: Applying Convolutional Filters

Implement a function ```R = applyImageFilter(I,F)``` that takes a
gray-scale image $I$ and a filter $F$ as inputs, and returns the result of the convolution
of the two.

Assume we are given a gray-scale image $I[x, y]$, of size $W \times H$, such that $0 \leq x \leq W-1$,
and $0 \leq y \leq H-1$. We want to apply a filter $F[i, j]$ to image $I$. The filter $F$ is of size $(2N + 1) \times (2M + 1)$, such that $−N \leq i \leq N$, and $−M \leq j \leq M$.

The result can be computed as

\begin{align}
R[x, y] = (I ∗ F)[x, y] = \sum_{i=-N}^{N} \sum_{i=-M}^{M} I[x − i, y − j]~F[i, j]
\end{align}


* Note: You can use your implemenation of ```applyImageFilter``` from EX2.
    -  To avoid numerical issues, make sure $I$ and $F$ are of type float.
    -  Apply zero-padding to make the filtered image having the same size as the input.

In [ ]:
def applyImageFilter(I, F):
    # First input parameter: I
    #     Input image. It should be a 2D matrix. According to the notation in the description, it has W rows and 
    #     H columns.
    # Second input parameter: F
    #     Filter used for the convolution. It should be a 2D matrix. According to the notation in the decription,
    #     it has (2N+1) rows and (2M+1) columns.
    # Third input parameter: padding mode
    #     This is an optional parameter. Default value will be same (i.e. if you only pass I, F parameters
    #     when calling the function, padding will be set to 'same'). The other padding mode is 'valid'
    #     See below in the code for the explanaiton of the two modes. 
    
    # If image is not of type float, convert it to float
    if not np.issubdtype(I.dtype,float):
        I = np.float64(I)
        
    # If the filter is not of type float, convert it to float
    if not np.issubdtype(F.dtype,float):
        F = np.float64(F)
    
    # Shape of Filter
    N_, M_ = F.shape
    
    # Check whether the dimensions of input are accurate, they should be odd
    if not N_%2: 
        raise ValueError('Number of rows in the filter must be odd')
        
    if not M_%2:
        raise ValueError('Number of columns in the filter must be odd')
    
    # Compute the values for N,M which is used in the above description. 
    N = np.int64((N_-1)/2)
    M = np.int64((M_-1)/2)
    
    
    # Shape of the input image
    W, H = I.shape

    # In this case, the output image size will have same dimensions as the input
    # image (thus the name 'SAME').
    # To achieve this, padding size is determined accordingly
    # The padding enables us to perform convolutions at the bordering pixels. 

    I = np.pad(I, ((N,N),(M,M)), mode='constant') # default value for padding is 0 in np.pad
    # Initialize output image. Note that the size of R is same as the input I
    R = np.zeros((W,H), dtype = np.float64)      
    
    # Output image size
    W_R, H_R = R.shape
    
    for x in range(W_R): # iterating through rows of the output image in the direction ↓
        for y in range(H_R): # iterating through columns of the output image in the direction →
            
            # compute the value for R[x, y] 
            # x is the row index
            # y is the column index  
            for i in range(-N,N+1):
                for j in range(-M,M+1):  
                    # I[x+M-j, y+N-i]: I is the padded iamge. 
                    #                 At the x,y iteration, (x,y) corresponds to R[x,y] pixel.
                    #                 This is the location that the filter is centered in this iteration.
                    #                 R[x,y] corresponds to I[x+M, y+N] pixel in I.
                    #                 With the center (x,y), we iterate through it's neighborhood [-N,N] in 
                    #                 x direction and [-M,M] in y direction. 
                    #                 This gives us the coordinates I[x+N-i, y+M-j]
                    # F[i+N,j+M]: F is the filter. In the definition above its indices ranges from 
                    #             -N to N in x direction and -M to M in y direction.
                    #             But the matrix indices are positive, therefore we shift the indices [i+N,j+M].
                    R[x,y] += I[x+N-i, y+M-j]*F[i+N,j+M]
                    
    return R

## Q3: Compute Derivative Images

Implement a function ```R = derivativeImage(I, fsize, fsigma)``` that takes a
gray-scale image $I$, and the size and the sigma of the gaussian kernel as inputs, 
and returns the magnitude image of the derivative of $I$.

\begin{align}
R = \sqrt{\frac{\partial I}{\partial x}^2 + \frac{\partial I}{\partial y}^2}
\end{align}


In [ ]:
def derivativeImage(I, fsize, fsigma):
    # -------------------------------
    # Implement your code here
    # -------------------------------
    
    # -------------------------------

    g = gaussian_filter(fsize, fsigma)
    g_d = gaussian_filter_derivative(fsize, fsigma)

    Ix = applyImageFilter(I, g)
    Ix = applyImageFilter(Ix, g_d.T)

    Iy = applyImageFilter(I, g.T)
    Iy = applyImageFilter(Ix, g_d)

    R = np.sqrt(Ix**2+Iy**2)
    return R

## Q4: Thresholding

Implement a function ```R = thresholding(I, threshold)``` that takes a derivative images $I$ and the threshold as inputs, and returns the thinned derivative image.

Hint: simply set pixel values below the threshold to 0.

In [ ]:
def thresholding(I, threshold):
    # -------------------------------
    # Implement your code here
    # -------------------------------
    
    # -------------------------------

    R = I.copy()
    R[I<threshold] = 0
    return R

In [ ]:
# -------------------------------
# Do not change this code 
# -------------------------------
fsize = 5
fsigma = 1
threshold = 20
I = cv2.imread('images/coins.png',0)

I_grad = derivativeImage(I, fsize, fsigma)
I_grad_thinned = thresholding(I_grad, threshold)

# Visualizing the results
fig = plt.figure(1, figsize=(10,15)) 
fig.add_subplot(1, 3, 1)
plt.imshow(I, cmap='gray')
plt.title('Original Image')

fig.add_subplot(1, 3, 2)
plt.imshow(I_grad,cmap='jet')
plt.colorbar(fraction=0.046, pad=0.04)
plt.title('Gradient magnitude')

fig.add_subplot(1, 3, 3)
plt.imshow(I_grad_thinned,cmap='jet')
plt.colorbar(fraction=0.046, pad=0.04)
plt.title('Thinned Gradient magnitude')
 
# -------------------------------